In [89]:
import re

In [90]:
def replace_number_if_question(string, replacement):
    # Use regular expression to find a number preceded by the word "Question"
    return re.sub(r'(?<=Question )\d+', str(replacement), string)

In [91]:
# Create a list of files to process
import os

# Get the current working directory
cwd = os.getcwd()
cwd = os.path.join(cwd, "orig")

def alphanumeric_key(s):
    return [int(c) if c.isdigit() else c.lower() for c in re.split(r'(\d+)', s)]

# List all files in the CWD that start with "questions-" and have a .qmd extension
files = [f for f in os.listdir(cwd) if f.startswith('q') and f.endswith('.qmd')]
files = sorted(files, key=alphanumeric_key)

In [92]:
files

['q0.qmd',
 'q10.qmd',
 'q20.qmd',
 'q30.qmd',
 'q40.qmd',
 'q50.qmd',
 'q60.qmd',
 'q70.qmd',
 'q80.qmd',
 'q90.qmd',
 'q100.qmd',
 'q110.qmd',
 'q120.qmd',
 'q130.qmd',
 'q140.qmd',
 'q150.qmd',
 'q160.qmd',
 'q170.qmd',
 'q180.qmd',
 'q190.qmd',
 'q200.qmd',
 'q210.qmd',
 'q220.qmd',
 'q230.qmd']

In [93]:
# Initialize an empty list to store the questions
questions = []


for file in files:
    # Create the full file path
    file_path = os.path.join(cwd, file)
    
    # Read the file and process the lines
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Initialize variables
    current_question = []
    is_question = False
    # Loop through the lines and extract questions
    for line in lines:
        if line.startswith('###'):
            line = replace_number_if_question(line, "%Q#%")
            # If we're already capturing a question, save the previous one
            if current_question:
                questions.append(''.join(current_question).strip())
                current_question = []
            
            # Start a new question capture
            is_question = True
            current_question.append(line)
        elif is_question:
            # Continue adding lines to the current question until we hit a new question
            current_question.append(line)

    # Add the last question to the list if any
    if current_question:
        questions.append(''.join(current_question).strip())

import random

# Randomize the order of questions
random.shuffle(questions)

In [94]:
for i, question in enumerate(questions):
    questions[i] = question.replace("%Q#%", str(i+1))

In [95]:
len(questions)

196

In [96]:
# Define the number of questions per file
questions_per_file = 10
includes = []
# Iterate over the questions list in chunks of 10
for i in range(0, len(questions), questions_per_file):
    # Create the file name based on the current chunk index
    file_name = f'q{i}.qmd'
    includes.append(f"{{{{< include q{i}.qmd >}}}}\n")
    # Get the current chunk of questions
    chunk = questions[i:i + questions_per_file]
    
    # Write the chunk to the file
    with open(file_name, 'w') as f:
        for question in chunk:
            f.write(question + '\n\n')

In [97]:
with open('review.qmd', 'w') as f:
    for include in includes:
        f.write(include)